In [1]:
import pandas
import numpy as np
from keras import *
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow import math
from plot_keras_history import plot_history

import pywt

In [3]:
data_frame = pandas.read_csv("bonn_dataset.csv")

labels = data_frame['y']
first_column = data_frame.columns[0]
data = data_frame.copy()
data.pop(first_column)
data.pop('y')

0        4
1        1
2        5
3        5
4        5
        ..
11495    2
11496    1
11497    5
11498    3
11499    4
Name: y, Length: 11500, dtype: int64

In [26]:
# print(data.shape[1])


filtered = []

for i in range(len(data)):
    # decompose the signal 5 levels
    vec = data.iloc[i]
    coefficients = pywt.wavedec(vec, 'db5', 5)
        
    filtered.append(np.array(coefficients))
    
filtered = np.array(filtered)
# print(filtered)

In [40]:
    
    
def rnn(shape):
    model = Sequential(name='rnn')

    inputs = Input(shape=shape)
    
    model.add(inputs)
    model.add(layers.Reshape(shape))
#     model.add(layers.Embedding(shape[0] + 1, 86))
    
    model.add(layers.GRU(64))
    
    model.add(layers.Dense(20, activation='sigmoid'))
    model.add(layers.Dense(5, activation='softmax'))
    
#     model.add(layers.Reshape((data.shape[1], 1)))
    # model.add(layers.Conv1D(64, 25, activation='relu'))
#     model.add(layers.Conv1D(32, 15, activation='relu'))
#     model.add(layers.Conv1D(16, 10, activation='relu'))
#     model.add(layers.Conv1D(8, 5, activation='relu'))
#     model.add(layers.MaxPooling1D())
#     model.add(layers.Flatten())
#     # model.add(layers.Dense(100, activation='sigmoid'))
#     model.add(layers.Dense(40, activation='sigmoid'))
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(5, activation='softmax'))
    
    return model
rnn((data.shape[1], 1)).summary()

Model: "rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 178, 1)            0         
_________________________________________________________________
gru_11 (GRU)                 (None, 64)                12864     
_________________________________________________________________
dense_38 (Dense)             (None, 20)                1300      
_________________________________________________________________
dense_39 (Dense)             (None, 5)                 105       
Total params: 14,269
Trainable params: 14,269
Non-trainable params: 0
_________________________________________________________________


In [41]:
x_train, x_test, y_train, y_test = train_test_split(data, pandas.get_dummies(labels, prefix='state'))

model = rnn((data.shape[1], 1))
model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=32, epochs=50)


Epoch 1/50
270/270 [==============================] - 23s 86ms/step - loss: 1.2680 - accuracy: 0.4690
Epoch 2/50
270/270 [==============================] - 23s 86ms/step - loss: 0.9502 - accuracy: 0.5941
Epoch 3/50
270/270 [==============================] - 23s 85ms/step - loss: 0.8471 - accuracy: 0.6279
Epoch 4/50
270/270 [==============================] - 24s 87ms/step - loss: 0.7775 - accuracy: 0.6602
Epoch 5/50
270/270 [==============================] - 23s 86ms/step - loss: 0.7365 - accuracy: 0.66810s - loss: 0.7376 - accuracy
Epoch 6/50
270/270 [==============================] - 24s 88ms/step - loss: 0.7142 - accuracy: 0.6780
Epoch 7/50
270/270 [==============================] - 24s 88ms/step - loss: 0.6825 - accuracy: 0.6881
Epoch 8/50
270/270 [==============================] - 24s 87ms/step - loss: 0.6862 - accuracy: 0.6878
Epoch 9/50
270/270 [==============================] - 23s 87ms/step - loss: 0.6604 - accuracy: 0.6933
Epoch 10/50
270/270 [==============================] -

In [42]:
model.evaluate(x_test, y_test)

90/90 [==============================] - 2s 17ms/step - loss: 0.6467 - accuracy: 0.7353


[0.6467121243476868, 0.7353043556213379]

In [100]:

# row.plot()
# pyplot.show()

filtered_data = []
wavelets = ['db6', 'sym6', 'coif2'] # all these wavelets produce coefficients of the same length

# row = data.iloc[0]
# ca, cd = pywt.dwt(row, 'coif2')
# print(len(cd))

for i in range(len(data)):
    row = data.iloc[i]
    
    new_row = []
    
    for wt in wavelets:
        ca, cd = pywt.dwt(row, wt)
        new_row.append(ca)
        new_row.append(cd)
    
    filtered_data.append(np.array(new_row))

filtered_data = np.array(filtered_data)  

In [101]:
# print(filtered_data)
def filtered_cnn(shape):
    model = Sequential(name='filtered')
    model.add(Input(shape=shape[1:]))
    model.add(layers.Reshape((shape[2], shape[1])))
    model.add(layers.Conv1D(32, 15, activation='relu'))
    model.add(layers.Conv1D(16, 10, activation='relu'))
    model.add(layers.Conv1D(8, 5, activation='relu'))
    model.add(layers.MaxPooling1D())
    model.add(layers.Flatten())
    # model.add(layers.Dense(100, activation='sigmoid'))
    model.add(layers.Dense(50, activation='sigmoid'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='softmax'))
    
    return model
          
filtered_cnn(filtered_data.shape).summary()

Model: "filtered"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_14 (Reshape)         (None, 94, 6)             0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 80, 32)            2912      
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 71, 16)            5136      
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 67, 8)             648       
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 33, 8)             0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 264)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 50)                132

In [103]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1337)

accuracy_fold = []
loss_fold = []

fold_nr = 0

for train_indexes, test_indexes in kf.split(filtered_data, labels):
    x_train = filtered_data[train_indexes]
    y_train_multilabel = labels.iloc[train_indexes]
    y_train = pandas.get_dummies(y_train_multilabel, prefix='state')
    
    
    x_test = filtered_data[test_indexes]
    y_test_multilabel = labels.iloc[test_indexes]
    y_test = pandas.get_dummies(y_test_multilabel, prefix='state')
    
    # define a new model
    model = filtered_cnn(filtered_data.shape)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    print("----------------------------------------------------------------------")
    print("Training for fold nr: {}".format(fold_nr))
    history = model.fit(x_train, y_train, batch_size=32, epochs=50, verbose=0)
    
    loss, accuracy = model.evaluate(x_test, y_test)
    
    print("Test Loss: {}".format(loss))
    print("Test Accuracy: {}".format(accuracy))
    
    accuracy_fold.append(accuracy)
    loss_fold.append(loss)
    
    fold_nr += 1
    print("----------------------------------------------------------------------")

# y = pandas.get_dummies(labels, prefix='state')
# # print(y)
# x_train, x_test, y_train, y_test = train_test_split(filtered_data, y, test_size=0.1, random_state=1337)
# model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(x_train, y_train, batch_size=32, epochs=40)
# loss, accuracy = model.evaluate(x_test, y_test)

----------------------------------------------------------------------
Training for fold nr: 0
36/36 [==============================] - 0s 2ms/step - loss: 1.3265 - accuracy: 0.3904
Test Loss: 1.3264912366867065
Test Accuracy: 0.39043477177619934
----------------------------------------------------------------------
----------------------------------------------------------------------
Training for fold nr: 1
36/36 [==============================] - 0s 2ms/step - loss: 1.1013 - accuracy: 0.5052
Test Loss: 1.1012675762176514
Test Accuracy: 0.5052173733711243
----------------------------------------------------------------------
----------------------------------------------------------------------
Training for fold nr: 2
36/36 [==============================] - 0s 2ms/step - loss: 0.9355 - accuracy: 0.5722
Test Loss: 0.9354562163352966
Test Accuracy: 0.5721738934516907
----------------------------------------------------------------------
------------------------------------------------

In [104]:
print("Average Loss for the model: {}".format(sum(loss_fold) / 10))
print("Average accuracy for the model: {}".format(sum(accuracy_fold) / 10))

Average Loss for the model: 1.0949480950832366
Average accuracy for the model: 0.5066956460475922
